# Cài đặt các gói cần thiết

In [7]:
%pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.1
    Uninstalling pip-24.1.1:
      Successfully uninstalled pip-24.1.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install onnx onnxscript onnxruntime torchsummary
# pip install onnx onnxscript để có thể chuyển model từ .pth hoặc .pt sang .onnx
# pip install onnxruntime để có môi trường chạy model định dạng tệp .onnx

Note: you may need to restart the kernel to use updated packages.


# Xây dựng model hoặc load vào model

In [4]:
%cd /mnt/Data/Projects/ThyroidCancerClassifier

/mnt/Data/Projects/ThyroidCancerClassifier


/home/harito/venv/pyvenv311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# model_file_path = '.../model.pt'
model_file_path = '/mnt/Data/Projects/h97_resnet_B2_B5B6_dataver1/best_h97_resnet_B2_B5B6_dataver1_model.pt' #...

In [27]:
# Đây là ví dụ cách load vào model H97_ResNet đã được train và lưu vào file có đường dẫn model_file_path
from src.model.classifier.H97 import H97_ResNet
import torch

torch_model = H97_ResNet(num_classes=2)
torch_model.load_state_dict(torch.load(model_file_path, map_location=torch.device('cpu')))
torch_model.eval()

/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


H97_ResNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

# Load model từ dạng của PyTorch sang format của ONNX

In [28]:
torch_input = torch.randn(1, 3, 224, 224)
# torch.onnx.dynamo_export(model, /, *model_args, export_options=None, **model_kwargs)
# num_classes = 2
# onnx_program = torch.onnx.dynamo_export(model=torch_model, model_args=(torch_input,))
# onnx_program = torch.onnx.export(torch_model, torch_input, "/mnt/Data/Projects/best_h97_resnet_B2_B5B6_dataver1_model.onnx", opset_version=18)

# Export the model to ONNX
try:
    onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)
    print("Model exported successfully using dynamo_export.")
except Exception as e:
    print(f"Failed to export the model using dynamo_export. Error: {e}")

/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torch/onnx/_internal/fx/passes/readability.py:53: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torch/fx/graph.py:1460: UserWarning: Node feature_extractor_1_running_mean target feature_extractor/1/running_mean feature_extractor/1/running_mean of  does not reference an nn.Module, nn.Parameter, or buffer, wh

Model exported successfully using dynamo_export.


# Lưu model ở format của ONNX thành file .onnx

In [29]:
# model_destination_path = '.../my_image_classifier.onnx'
model_destination_path = '/mnt/Data/Projects/best_h97_resnet_B2_B5B6_dataver1_model.onnx' #...

In [30]:
onnx_program.save(model_destination_path)

# Load lại model từ file .onnx

In [31]:
import onnx
onnx_model = onnx.load(model_destination_path)
onnx.checker.check_model(onnx_model)

# Trực quan kiến trúc model (lưu ở định dạng tệp .onnx) với [Netron](https://netron.app/)

Truy cập trang web [sau đây](https://netron.app/). Sau đó chọn tệp tin .onnx và xem hình ảnh trực quan ra.

# Chạy 1 model .onnx với môi trường onnxruntime

In [32]:
import onnxruntime

torch_input = torch.randn(
    1, 3, 224, 224
)  # sửa lại kích thước input cho phù hợp với model của mình
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

# Tạo 1 session để suy luận model
# tại đường dẫn model_destination_path
# với môi trường CPUExecutionProvider hoặc CUDAExecutionProvider
# Kiểm tra xem có GPU hỗ trợ ONNX Runtime không
if "CUDAExecutionProvider" in onnxruntime.get_available_providers():
    providers = ["CUDAExecutionProvider"]
else:
    providers = ["CPUExecutionProvider"]

# Tạo phiên suy luận sử dụng thiết bị phù hợp
ort_session = onnxruntime.InferenceSession(
    model_destination_path, providers=providers
)


def to_numpy(tensor):
    """
    Định nghĩa một hàm để chuyển tensor PyTorch sang mảng NumPy.
    Điều này cần thiết vì ONNX Runtime sử dụng mảng NumPy làm định dạng dữ liệu đầu vào.
    Nếu đang sử dụng GPU, tensor sẽ được chuyển về CPU trước khi chuyển sang NumPy.
    """
    if "CUDAExecutionProvider" in onnxruntime.get_available_providers():
        tensor = tensor.cuda()
    return (
        tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )


# Tạo một từ điển của dữ liệu đầu vào cho ONNX Runtime,
# chuyển đổi tensor đầu vào PyTorch sang mảng NumPy và
# gán chúng với tên của các đầu vào mô hình như được xác định trong phiên suy luận ONNX.
onnxruntime_input = {
    k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)
}

# Thực hiện suy luận với dữ liệu đầu vào đã được chuẩn bị,
# None ở đây chỉ ra rằng chúng ta muốn lấy tất cả đầu ra từ mô hình.
# Kết quả suy luận được lưu trong onnxruntime_outputs.
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

Input length: 1
Sample input: (tensor([[[[ 9.8843e-01, -8.7964e-01,  7.6984e-01,  ...,  2.4357e-01,
            2.0037e-01, -7.2967e-01],
          [ 1.5482e+00,  5.4385e-01, -2.3133e-01,  ..., -6.3933e-01,
            5.7336e-01, -1.2709e+00],
          [ 8.4850e-01, -5.0144e-01,  8.7841e-01,  ..., -1.5794e-01,
            4.4783e-01, -3.8907e-01],
          ...,
          [-1.4656e+00,  1.6186e-01,  7.4490e-01,  ..., -3.5898e-01,
            6.0841e-01,  4.6724e-01],
          [ 8.8117e-02,  1.2423e+00,  2.1215e+00,  ..., -1.9583e+00,
           -1.1718e-01, -6.5420e-01],
          [-1.5409e+00,  1.6645e-01, -6.8965e-01,  ..., -6.7262e-01,
            4.0382e-01, -4.3762e-01]],

         [[-1.7549e-01, -1.4687e+00,  1.0858e+00,  ..., -8.3161e-02,
            8.7542e-01,  1.2830e+00],
          [-1.1562e+00, -2.7096e-01,  6.8487e-01,  ..., -3.7020e-01,
            2.0894e+00,  1.3889e-03],
          [-4.3982e-01, -4.5690e-01, -4.4928e-01,  ...,  2.0361e-01,
           -1.8560e+00, -1.

# So sánh kết quả chạy bởi model PyTorch với model .onnx có giống nhau không

In [33]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[ 4.271691 , -5.1230164]], dtype=float32)]
